In [ ]:
rostopic echo /gazebo/model_states -n1

http://docs.ros.org.ros.informatik.uni-freiburg.de/hydro/api/ric_mc/html/namespacemembers.html#index_n

# Unit 3: Services in ROS

## Part 1

What will you learn with this unit?

1. What a **service** is
2. How to **manage services of a robot**
3. How to **call a service**

To understand what services are and when to use them, you have to compare them with topics and actions.



Imagine you have your own personal BB-8 robot. It has a laser sensor, a face-recognition system, and a navigation system.
> * The laser will use a Topic to publish all of the laser readings at 20hz. 
> * We use a topic because we need to have that information available all the time for other ROS systems, such as the navigation system.

The Face-recognition system will provide a Service. 
> * Your ROS program will call that service and **WAIT** until it gives you the name of the person BB-8 has in front of it.

The navigation system will provide an **Action**. 
> * Your ROS program will call the action to move the robot somewhere, and **WHILE** it's performing that task, your program will perform other tasks, such as complain about how tiring C-3PO is. 
> * And that action will give you Feedback (for example: distance left to the desired coordinates) along the process of moving to the coordinates.

## **So... What's the difference between a Service and an Action?**

> * **Services** are **Synchronous**. 
>> When your ROS program calls a service, your program can't continue until it receives a result from the service.

> * **Actions** are **Asynchronous**.
>>  It's like launching a new thread. When your ROS program calls an action, your program can perform other tasks while the action is being performed in another thread.

**Conclusion: Use services when your program can't continue until it receives the result from the service.**

## **1. Services Introduction**


## **Example 3.1**



In [ ]:
roslaunch iri_wam_aff_demo start_demo.launch

The launch file has launched two nodes (Yes! You can launch more than one node with a single launch file):

> * /iri_wam_reproduce_trajectory
> * /iri_wam_aff_demo


> 1. The first node provides the **/execute_trajectory** service. This is the node that contains the service.
> 2. The second node, performs calls to that **service**. When the service is called, the robot will execute a given trajectory.

**You have to leave the start_demo.launch running, otherwise the services won't be there to see them.**

### **1. listing all the services available**

In [ ]:
rosservice list

In [ ]:
/camera/rgb/image_raw/compressed/set_parameters
/camera/rgb/image_raw/compressedDepth/set_parameters
/camera/rgb/image_raw/theora/set_parameters
/camera/set_camera_info
/camera/set_parameters
/execute_trajectory
/gazebo/apply_body_wrench
..

### **2. You can get more information about any service by issuing the following command:**

In [ ]:
rosservice info /name_of_your_service

In [ ]:
rosservice info /execute_trajectory



```
user ~ $ rosservice info /execute_trajectory
Node: /iri_wam_reproduce_trajectory
URI: rosrpc://ip-172-31-17-169:35175
Type: iri_wam_reproduce_trajectory/ExecTraj
Args: file
```



4. **Type**: It refers to the kind of message used by this service. It has the same structure as topics do.
> * It's always made of **package_where_the_service_message_is_defined / Name_of_the_File_where_Service_message_is_defined**. 
> * In this case, the package is **iri_wam_reproduce_trajectory**, and the file where the Service Message is defined is called **ExecTraj**.

5. **Args**: Here you can find the arguments that this service takes when called. 
> * In this case, it only takes a **trajectory file path** stored in the variable called **file.**

## **Example 3.3**



In [ ]:
roscd iri_wam_aff_demo
cd launch/
cat start_demo.launch

1) The first part of the launch file calls another launch file called **start_service.launch**.


3) The second node is not a Python node, but a cpp compiled (binary) one.
> You can build ROS programs either in Cpp or Python. This course focuses on Python.

In [ ]:
<launch>

# 1. Provides service 
  <include file="$(find iri_wam_reproduce_trajectory)/launch/start_service.launch"/>

# 2. There is no .py because it's cpp file 
  <node pkg ="iri_wam_aff_demo"
        type="iri_wam_aff_demo_node"
        name="iri_wam_aff_demo"
        output="screen">
  </node>

</launch>

## **2. How to call a service**

You can **call a service manually** from the console.




In [ ]:
rosservice call /the_service_name TAB-TAB

## **Example 3.4**

Let's call the service with the name **/trajectory_by_name** by issuing the following command. 
> * But before being able to call this Service, you will have to launch it.
> * For doing so you can execute the following command:



In [ ]:
# 1. 

roslaunch trajectory_by_name start_service.launch

In [ ]:
# 2. 

rosservice call /trajectory_by_name [TAB]+[TAB]


**"traj_name: 'the_name_of_the_trajectory_you_want_to_execute'"**

Use that service to execute one trajectory with the WAM Arm. 
> The trajectories that are available are the following ones: 
1. init_pos, 
2. get_food and 
3. release_food.

In [ ]:
rosservice call /trajectory_by_name "traj_name: 'get_food'"

## **3**. But how do you interact with a service programatically?

**Python Program {3.5}: simple_service_client.py**



In [ ]:
#! /usr/bin/env python

import rospy

# Import the service message used by the service /trajectory_by_name
from trajectory_by_name_srv.srv import TrajByName, TrajByNameRequest
import sys

# 1. Initialise a ROS node with the name service_client
rospy.init_node('service_client')

# 2. Wait for the service client /trajectory_by_name to be running
# if available then works forwrd 
rospy.wait_for_service('/trajectory_by_name')

# 3. Create the connection to the service,
#  name of msg type
traj_by_name_service = rospy.ServiceProxy('/trajectory_by_name', TrajByName)

# 4. Create an object of type TrajByNameRequest
                                             # like var= Twist()                                          
traj_by_name_object = TrajByNameRequest()

# 5. Fill the variable traj_name of this object with the desired value
                                               # like var.data= 53
traj_by_name_object.traj_name = "release_food"

# 6. Send through the connection the name of the trajectory to be executed by the robot
result = traj_by_name_service(traj_by_name_object)

# Print the result given by the service called
print result

```
success: True
status_message: "Successfully executed trajectory"
```



## **4**. How to know the structure of the service message used by the service?





In [ ]:
rosservice info /name_of_the_service

In [ ]:
rossrv show name_of_the_package/Name_of_Service_message

In [ ]:
#1. service info 
rosservice info /trajectory_by_name



```
Node: /traj_by_name_node
URI: rosrpc://rosdscomputer:36457
Type: trajectory_by_name_srv/TrajByName
Args: traj_name
```



In [ ]:
# 2. Service service data 

rossrv show trajectory_by_name_srv/TrajByName


 **TrajByName.srv**

```
string traj_name
---
bool success
string status_message
```



It should, because it's the same structure as the Topics messages, with some addons.



**1. Service message files have the extension .srv**
> Remember that Topic message files have the extension .msg

**2. Service message files are defined inside a srv directory.**
> Remember that Topic message files are defined inside a msg directory.

**3. Service messages have TWO parts:**
```
REQUEST     # HOW you will do a call** to your service. 

---

RESPONSE    #HOW your service will respond after completing its functionality
```


> 1. In the case of the TrajByName service,
>> * **REQUEST** contains a string called **traj_name** 
>> * **RESPONSE** is composed of a boolean named **success**,
>> * and a string named **status_message**.

> 2. The Number of elements on each part of the service message can vary depending on the service needs. 
> * You can even put none if you find that it is irrelevant for your service. 
> * **The important part of the message is the three dashes**
```
---
```
> **because they define the file as a Service Message.**

**Summarizing:**

The **REQUEST** is the part of the service message that defines **HOW you will do a call** to your service. 
> * This means, what variables you will have to pass to the Service Server so that it is able to complete its task.

The **RESPONSE** is the part of the service message that defines **HOW your service will respond after completing its functionality**. 
> * If, for instance, it will return an string with a certaing message saying that everything went well, or if it will return nothing, etc...

### **4. How to use Service messages In your code:**

**Whenever a Service message is compiled**, 
> **3 message objects are created**. 
>> * Let's say have a Service message named **MyServiceMessage**. 
>> * If we compile this message, 3 objects will be generated:


> * **1. MyServiceMessage**: This is the Service message itself. 
>> It's used for creating a connection to the service server, as you saw in Python Program {3.5}:

In [ ]:
traj_by_name_service = rospy.ServiceProxy('/trajectory_by_name', TrajByName)

> **2. MyServiceMessageRequest**: This is the object used for creating a request to send to the server. 
>> * Just like your web browser (a client) connects to a webserver to request web pages, using HttpRequest objects.
>> *  So, this object is used for sending a request to the service server, as you saw in Python Program {3.5}:

In [ ]:
traj_object = TrajByNameRequest()
# Fill the variable traj_name of this object with the desired value

traj_object.traj_name = "release_food"
# Send through the connection the name of the trajectory to be executed by the robot

result = traj_by_name_service(traj_object)

> **3. MyServiceMessageResponse**: This is the object used for sending a response from the server back to the client, whenever the service ends.


## Exercise 3.1

In [ ]:
catkin_create_pkg unit_3_services rospy iri_wam_reproduce_trajectory

In [ ]:
cd unit_3_services

In [ ]:
mkdir launch

In [ ]:
<launch>

  <include file="$(find iri_wam_reproduce_trajectory)/launch/start_service.launch"/>

  <!-- Here will go our python script that calls the execute_trajectory service -->

</launch>

In [ ]:
roslaunch unit_3_services my_robot_arm_demo.launch


```
[ WARN] [1506621085.854497817, 4552.117000000]: Shutdown request received.
[ WARN] [1506621085.854552030, 4552.117000000]: Reason given for shutdown: [new node registered with same name]
[ERROR] [1506621085.965156755, 4552.207000000]: Failed to execute [get_food.txt] trajectory
```



In [ ]:
rosservice list | grep execute_trajectory

In [ ]:
user:~/catkin_ws$ rosservice list | grep execute_trajectory
/execute_trajectory

In [ ]:
rosservice info /execute_trajectory


```
user:~/catkin_ws$ rosservice info /execute_trajectory
Node: /iri_wam_reproduce_trajectory
URI: rosrpc://ip-172-31-1-3:57245
Type: iri_wam_reproduce_trajectory/ExecTraj
Args: file
```



In [ ]:
rossrv show iri_wam_reproduce_trajectory/ExecTraj



```
user:~/catkin_ws$ rossrv show iri_wam_reproduce_trajectory/ExecTraj
string file
---
```



As you can see this ServiceMessage called ExecTraj has:

**Request**: only a variable called **file**, of type **String**. String is a basic message type that yo can find in the **std_msgs** package.

**Response**: In this case it has none. So no response will be given when this service is called. It will be what we know as **Empty** response.

Now that we have all the information about the service we can start creating the python program to call it.

> * Make the arm robot move following a trajectory specified in a file.
> * Modify the previous code of example 3.5 that called **/trajectory_by_name**, to call the /execute_trajectory service instead. 

> The new python file could be called **exercise_3_1.py** for future reference.

### exercise_3_1.py 

In [ ]:
#! /usr/bin/env python

import rospy
from iri_wam_reproduce_trajectory.srv import ExecTraj, ExecTrajRequest # Import the service message used by the service /execute_trajectory

rospy.init_node('service_execute_trajectory_client') # Initialise a ROS node with the name service_client
rospy.wait_for_service('/execute_trajectory') # Wait for the service client /execute_trajectory to be running

# Create the connection to the service
execute_trajectory_service_client = rospy.ServiceProxy('/execute_trajectory', ExecTraj) 
# Create an object of type ExecTrajRequest
execute_trajectory_request_object = ExecTrajRequest() 

"""
user:~/catkin_ws$ rossrv show iri_wam_reproduce_trajectory/ExecTraj
string file
---

"""
 # Fill the variable file of this object with the desired file path
execute_trajectory_request_object.file = "file_path"

# Send through the connection the path to the trajectory file to be executed
result = execute_trajectory_service_client(execute_trajectory_request_object)

print result # Print the result type ExecTrajResponse



In [ ]:
roscd iri_wam_reproduce_trajectory

In [ ]:
cd config

In [ ]:
ls



```
user:/opt/ros/indigo/share/iri_wam_reproduce_trajectory/config$ ls
get_food.txt  init_pos.txt  release_food.txt
```



In [ ]:
roscd unit_3_services
mkdir config
roscd iri_wam_reproduce_trajectory/config
cp get_food.txt /home/user/catkin_ws/src/unit_3_services/config

In [ ]:
0.0112138,0.942628,0.133408,1.65916,-0.214736,-1.05983,-0.430974,0.00213521,0.313696,0.0892853,-0.29252,-0.110414,0.0677202,0.140693,-0.00383239,-0.0665293,0.0935825,-0.27529,-0.393899,0.351521,-0.0444471,0.579301,2.77829,-2.87128,0.0812354,-0.336139,-0.000311573,0.0183247,-1.86726,1.48766,3.67008,-1.55314,0.302464,0.355123,0.0429483,0.0256781,-0.100288,0.0139974,-0.00673008,-0.00101568,-0.000844474,-1.56452e-05,-0.687394,-2.01554,2.78754,0.84521,-0.523724,0.667854,0.0283819
0.0112557,0.948886,0.135215,1.65325,-0.217031,-1.0584,-0.428172,0.0020552,0.31212,0.0911877,-0.297971,-0.118286,0.0750198,0.139555,-0.00413107,-0.0884723,0.0962923,-0.269863,-0.392985,0.375216,-0.0669714,0.566718,2.75946,-2.81956,0.0580163,-0.300554,-0.00282943,-0.0253767,-1.86951,1.50267,3.68917,-1.56147,0.306648,0.359253,0.0428217,0.0280618,-0.133746,0.0155503,-0.010118,-0.00110025,-0.0010578,-1.80724e-05,-0.550031,-3.1407,2.94782,-2.77195,-1.55137,0.36242,-0.0223026
0.0112958,0.955107,0.13706,1.64723,-0.219483,-1.05681,-0.425398,0.00196969,0.31013,0.0931383,-0.303284,-0.126112,0.0827353,0.137977,-0.00438779,-0.108162,0.0984662,-0.261782,-0.38954,0.393589,-0.088439,0.591821,2.68984,-2.77308,0.0552174,-0.250296,-0.0489743,0.0163657,-1.8719,1.51752,3.70853,-1.56945,0.311127,0.363222,0.042624,0.0266758,-0.161783,0.0161316,-0.0108963,-0.00112793,-0.00122445,-2.09172e-05,-1.45157,-3.65336,2.54961,-3.98732,-0.158789,0.0574192,0.0163606
0.0113343,0.961284,0.138945,1.64111,-0.22209,-.....

Its a sequence of joint values that the service **execute_trajectory** will read based on the path given in the call.

###  **Python File: Update exercise_3_1.py**

In [ ]:
import rospy
import rospkg
from iri_wam_reproduce_trajectory.srv import ExecTraj, ExecTrajRequest

rospy.init_node('service_execute_trajectory_client') # Initialise a ROS node 
rospy.wait_for_service('/execute_trajectory') 

execute_trajectory_service_client = rospy.ServiceProxy('/execute_trajectory', ExecTraj)
execute_trajectory_request_object = ExecTrajRequest() 

"""
user:~/catkin_ws$ rossrv show iri_wam_reproduce_trajectory/ExecTraj
string file
---

"""

rospack = rospkg.RosPack()
trajectory_file_path = rospack.get_path('iri_wam_reproduce_trajectory') + "/config/get_food.txt"

execute_trajectory_request_object.file = trajectory_file_path

# Send through the connection the path to the trajectory file to be executed
result = execute_trajectory_service_client(execute_trajectory_request_object)


print result # Print the result type ExecTrajResponse


In [ ]:
import rospy
import rospkg
from iri_wam_reproduce_trajectory.srv import ExecTraj, ExecTrajRequest

rospy.init_node('service_execute_trajectory_client') # Initialise a ROS node 
rospy.wait_for_service('/execute_trajectory') 

                                        # conncetion to service
service_client = rospy.ServiceProxy('/execute_trajectory', ExecTraj)
                                        # object of type ExecTrajRequest
request_object = ExecTrajRequest() 


rospack = rospkg.RosPack()
file_path = rospack.get_path('iri_wam_reproduce_trajectory') + "/config/get_food.txt"

request_object.file = file_path

result = service_client(request_object)
print result 


In [ ]:
<launch>

  <include file="$(find iri_wam_reproduce_trajectory)/launch/start_service.launch"/>

  <!-- Here will go our python script that calls the execute_trajectory service -->
    <node pkg ="unit_3_services"
        type="exercise_3_1.py"
        name="service_execute_trajectory_client"
        output="screen">
  </node>
  
  
</launch>